In [ ]:
!pip install transformers sentencepiece

In [49]:
import json
import random

import torch
from transformers import AutoTokenizer, MBartForConditionalGeneration, AutoModelForCausalLM

import pandas as pd

In [24]:
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [25]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [26]:
test_records = read_gazeta_records("gazeta_test.txt")

In [27]:
article_text = test_records[0]['text']

In [28]:
summary = dict()
summary['original'] = test_records[0]['summary']

####**MODEL "mbart_ru_sum_gazeta"**

In [29]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).cuda()

In [31]:
input_ids = tokenizer.prepare_seq2seq_batch(
    [article_text],
    src_lang="ru_XX", 
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=600
)["input_ids"].cuda()

output_ids = model.generate(
    input_ids=input_ids,
    max_length=162,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0
)[0]

summary['mbart_ru_sum_gazeta'] = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


####**MODEL "rugpt3medium_sum_gazeta"**

In [34]:
model_name = "IlyaGusev/rugpt3medium_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False, 
    padding=False,
    truncation=True
)["input_ids"]
input_ids = text_tokens + [tokenizer.sep_token_id]
input_ids = torch.LongTensor([input_ids])

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)

s = tokenizer.decode(output_ids[0], skip_special_tokens=False)
s = s.split(tokenizer.sep_token)[1]
summary['rugpt3medium_sum_gazeta'] = s.split(tokenizer.eos_token)[0]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 800 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


####**SUMMARY**

In [50]:
article_text

'Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел поможет раскрыть секреты о том, как они и им подобные объекты образовались в космосе». Каждый проект из выбранн

In [47]:
pd.set_option('display.max_colwidth', None)

In [48]:
pd.DataFrame(summary.items())

,0,1
0,original,"В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна."
1,mbart_ru_sum_gazeta,"В NASA назвали названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта."
2,rugpt3medium_sum_gazeta,"Американское космическое агентство NASA объявило имена четырех проектов, которые в ближайшее время могут быть отобраны для реализации. В их число вошли миссии, направленные на изучение Венеры, Марса, Юпитера и Сатурна."


**Предобученные модели суммаризируют вполне неплохо.**